### Pulling geo data from the list and displaying on a map


### read where.data, parsing geo data and create sqlite file

In [1]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")
 


Found in database  AGH University of Science and Technology

Found in database  Academy of Fine Arts Warsaw Poland

Found in database  American University in Cairo

Found in database  Arizona State University

Found in database  Athens Information Technology

Found in database  BITS Pilani

Found in database  Babcock University

Found in database  Banaras Hindu University

Found in database  Bangalore University

Found in database  Baylor University

Found in database  Beijing normal university

Found in database  Belarusian State University

Found in database  Belgrade University

Found in database  Beloit College

Found in database  Belorussian State University

Found in database  Ben Gurion University

Found in database  Bharthidasan University

Found in database  Boston University

Found in database  California Polytechnic State University of San Luis Obispo

Found in database  California State University San Bernardino

Found in database  City of Westminster College

Found in dat

#check for Datas 

In [2]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
##print("Open where.html to view the data in a browser")



aleja Adama Mickiewicza 30, 30-059 Kraków, Poland 50.06688579999999 19.9136192
Krakowskie Przedmieście 5, 00-068 Warszawa, Poland 52.2394019 21.0150792
Admin building AUC قسم أول القاهرة الجديدة، محافظة القاهرة‬ 4728120،، New Cairo 1, Cairo Governorate 4728120, Egypt 30.0201508 31.49908139999999
Tempe, AZ 85281, USA 33.4242399 -111.9280527
Monumental Plaza, Building C, 1st Floor, Leof. Kifisias 44, Marousi 151 25, Greece 38.0399391 23.8030901
Vidya Vihar, Pilani, Rajasthan 333031, India 28.3588163 75.58802039999999
121103, Ilishan-Remo, Ogun State, Nigeria 6.8920758 3.7181452
Ajagara, Varanasi, Uttar Pradesh 221005, India 25.2677203 82.99125819999999
Mysore Rd, Jnana Bharathi, Bengaluru, Karnataka 560056, India 12.9504048 77.5020617
1311 S 5th St, Waco, TX 76706, USA 31.5500848 -97.11354
19 Xin Wai Da Jie, Beitaipingzhuang, Hai Dian Qu, Bei Jing Shi, China, 100875 39.9619537 116.3662615
prasp. Niezaliežnasci 4, Minsk, Belarus 53.8930049 27.545623
Studentski trg 1, Beograd, Serbia 44.81

### display the result

In [3]:
import IPython
IPython.display.HTML(filename='where.html')